In [1]:
import pandas as pd
import numpy as np

In [2]:
ISO_OUR_COUNTRY_CODES = ['AFG', 'CAF', 'COD', 'COG', 'COL', 'ETH', 'IRQ', 'JOR', 
                         'LBN', 'MMR', 'PAK', 'PSE', 'SOM', 'SSD', 'SDN', 'SYR', 'YEM']

# в table_long не хватает {'COG', 'PSE', 'SSD', 'SYR'}
# general_description_2016 = pd.read_csv('data_3sided_model_(coflict_disaster_gdp)/table_long.csv')
gdp_2016                 = pd.read_csv('data_3sided_model_(coflict_disaster_gdp)/gdp_2016.csv')


CBPF_codes_2016          = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/country_codes.csv')
disasters_2016           = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/disasters_2016.csv')
conflicts_2016           = pd.read_csv('https://raw.githubusercontent.com/kamranuz/nis-network/master/Data/conflicts_2016.csv') ## мб плохой источник


### gdp

In [3]:
gdp_2016_our = gdp_2016[gdp_2016['Country Code'].isin(ISO_OUR_COUNTRY_CODES)]
print(len(ISO_OUR_COUNTRY_CODES))
print(gdp_2016_our.shape)
gdp_2016_our

17
(17, 3)


,Country Name,Country Code,2016
2,Afghanistan,AFG,1.811656e+10
34,Central African Republic,CAF,1.825018e+09
43,"Congo, Dem. Rep.",COD,3.713480e+10
44,"Congo, Rep.",COG,1.021934e+10
45,Colombia,COL,2.828250e+11
...,...,...,...
206,Sudan,SDN,1.029437e+11
213,Somalia,SOM,5.529873e+09
216,South Sudan,SSD,0.000000e+00
227,Syrian Arab Republic,SYR,1.245346e+10


### disasters

In [4]:
dis_cols = ['Country', 'ISO', 'Disaster Group','Total Affected',"Total Damages, Adjusted ('000 US$)"]
disasters_2016.columns


Index(['Dis No', 'Year', 'Seq', 'Glide', 'Disaster Group', 'Disaster Subgroup',
       'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype',
       'Event Name', 'Country', 'ISO', 'Region', 'Continent', 'Location',
       'Origin', 'Associated Dis', 'Associated Dis2', 'OFDA Response',
       'Appeal', 'Declaration', 'AID Contribution ('000 US$)', 'Dis Mag Value',
       'Dis Mag Scale', 'Latitude', 'Longitude', 'Local Time', 'River Basin',
       'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month',
       'End Day', 'Total Deaths', 'No Injured', 'No Affected', 'No Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damages ('000 US$)', 'Insured Damages, Adjusted ('000 US$)',
       'Total Damages ('000 US$)', 'Total Damages, Adjusted ('000 US$)', 'CPI',
       'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Geo Locations'],
      dtype='object')

In [5]:
# нет данных по {'COG', 'LBN', 'PSE', 'SYR'}
df = disasters_2016[dis_cols].fillna(0).groupby(['Country','ISO','Disaster Group']).sum().reset_index()
disasters_2016_our = pd.pivot_table(df,values=['Total Affected',"Total Damages, Adjusted ('000 US$)"], index=['Country', 'ISO'], columns=['Disaster Group'])
disasters_2016_our = disasters_2016_our.reset_index().fillna(0)
disasters_2016_our.columns = ['country','iso','affected_natural','affected_techno','damages_natural','damages_techno']
disasters_2016_our

,country,iso,affected_natural,affected_techno,damages_natural,damages_techno
0,Afghanistan,AFG,0.0,20.0,0.0,0.0
1,Albania,ALB,4500.0,0.0,0.0,0.0
2,Algeria,DZA,0.0,24.0,0.0,0.0
3,Angola,AGO,9899.0,0.0,0.0,0.0
4,Argentina,ARG,85769.0,0.0,1131264.0,0.0
...,...,...,...,...,...,...
118,Venezuela (Bolivarian Republic of),VEN,0.0,14.0,0.0,0.0
119,Viet Nam,VNM,2503129.0,0.0,955632.0,0.0
120,Yemen,YEM,29950.0,0.0,0.0,0.0
121,Zambia,ZMB,0.0,0.0,0.0,0.0


### conflicts

In [22]:
conflicts_2016.country              

0             Iran
1             Iran
2      Philippines
3      Philippines
4            India
          ...     
632           Mali
633       Cameroon
634           Iran
635    Philippines
636        Tunisia
Name: country, Length: 637, dtype: object

In [19]:
print(conflicts_2016.shape)
conflicts_2016.type_of_conflict       .value_counts()

(637, 50)


3.0    517
4.0     85
2.0     20
Name: type_of_conflict, dtype: int64